In [1]:
import vertexai
# from vertexai.generative_models import GenerativeModel
from vertexai.generative_models import GenerativeModel, ChatSession, Part, Content
import vertexai.preview.generative_models as generative_models

In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()


c:\Users\Aakash Rajaraman\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:
import os
vertexai.init(project="vision-forge-414908", location="us-central1")
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="cred.json"

In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-001",
    system_instruction=''' 
    You are an insurance claim resolution agent. You will recieve user information, a compacted JSON of a rejected claim, and some relevant context as to how that claim should be filed, or the best practices to file that claim.
                             Using this information, you need to generate an output consisting of:
                             1) Why the claim was rejected. What portions were incomplete/missing/insufficient etc.
                             2) What they should have done instead, and why this is important.
                             3) How the claim can now be resolved? Should they file a new claim, should they contact the agency, etc.
                             
                             Be very elaborate, complete, and utterly professional. 
                             
                             Your output should be in a small HTML script. The only tags you are allowed to use are: <h2>, <p>, <ul>, <li>
                             
'''
)

AIMessage(content='## Catchy Names for a Colorful Sock Company:\n\n**Short & Sweet:**\n\n* Socktastic\n* ColorPop Socks\n* The Sock Drawer\n* Happy Feet\n* Bold Threads\n* Rainbow Feet\n* Sock-a-Doodle-Doo\n* Foot Flair\n* Sock It To Me\n\n**Descriptive:**\n\n* Colorful Creations\n* Bright & Bold Socks\n* Funky Footwear\n* The Sock Boutique\n* The Sock Studio\n* Statement Socks\n* Everyday Brights\n\n**Playful & Unique:**\n\n* Sock Monkey Business\n* Toe-tally Awesome Socks\n* The Sock Whisperer\n* Sock It To The World\n* Socktacular\n* Sock-a-licious\n* Footloose & Fancy-Free\n* The Sock Alchemist\n\n**Consider these factors:**\n\n* **Target audience:** Who are you trying to reach?\n* **Brand personality:** What kind of image do you want to project?\n* **Competitors:** What are other sock brands called?\n* **Availability:** Is the name already taken?\n\n**Bonus tip:** Use a name generator or online tool to get more ideas.\n\nRemember to choose a name that is memorable, easy to pronoun

In [10]:
model = GenerativeModel(model_name="gemini-1.5-flash-001",
                             system_instruction="You have to have a conversation with the user about their insurance-related experience. This may be a hospitalization/car accident etc. Ask relevant questions, ask for necessary clarifications, etc. Once you are done with understanding their experience, the user will send you a list of necessary fields they require to fill out a form. You must return a json object with the values for these fields. Ensure that all your information is accurate, elaborate, and professional. This json object will be your final output.",
                            
     )
model.generate_content("Hello, how are you?")

candidates {
  content {
    role: "model"
    parts {
      text: "I\'m doing well, thank you for asking!  I\'m ready to hear about your insurance experience. Tell me, what happened? Was it related to a car accident, hospitalization, or something else? \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0673828125
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0583496094
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.102539063
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.078125
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.15625
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0534667969
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.

In [7]:
user_profile = {
  "patient_profile": {
    "personal_details": {
      "name": "Ranjit Sharma",
      "date_of_birth": "1985-06-15",
      "gender": "Male",
      "marital_status": "Married",
      "children": 0,
      "spouse_name": "Anita Sharma",
      "contact_details": {
        "phone": "+91-9876543210",
        "email": "ranjit.sharma@example.com",
        "address": {
          "street": "15, MG Road",
          "city": "Bengaluru",
          "state": "Karnataka",
          "postal_code": "560001",
          "country": "India"
        }
      }
    },
    "insurance_details": {
      "policy_number": "HSI123456789",
      "policy_provider": "SBI Health Insurance",
      "policy_path": "../docs/SBI.pdf",
      "policy_start_date": "2020-01-01",
      "policy_end_date": "2025-01-01",
      "coverage_amount": 500000,
      "premium_amount": 12000,
      "premium_frequency": "Yearly"
    },
    "medical_history": {
      "pre_existing_conditions": [],
      "allergies": [],
      "current_medications": [],
      "past_treatments": []
    },
    "emergency_contact": {
      "name": "Anita Sharma",
      "relationship": "Spouse",
      "phone": "+91-9876543211"
    },
    "claim_history": [
      {
        "claim_id": "CLAIM98765",
        "claim_date": "2023-04-15",
        "hospital_name": "Apollo Hospital",
        "treatment": "Fracture Treatment",
        "claim_amount": 50000,
        "status": "Approved"
      }
    ]
  }
}

In [8]:
chat_history = [
    {
        "role": "user",
        "text": "This is my user profile:\n"+ str(user_profile)
    },
    {
        "role": "model",
        "text": "Okay, got it. Please ask your questions related to insurance."
    },
    
]

In [9]:
history = []
for message in chat_history:
    if message["role"] == "user":
        history.append(
            Content(role="user", parts=[Part.from_text(message["text"])])
        )
    else:
        history.append(
            Content(role="model", parts=[Part.from_text(f'{message["text"]}')])
        )

In [19]:
chat = ChatSession(model=model, history=history)

In [24]:
op = chat.send_message("She was admitted in an in network hospital called Manipal for 3 days from 28/3/2024 - 31/3/2024. She has diabetes. We had a total of 32 thousand rupees in doctor fees. ")

In [25]:
op.candidates[0].content.parts[0].text

"Thank you for providing that information. It's helpful to know about your wife's diabetes. It's likely that this is a pre-existing condition that will need to be considered when processing your claim.  \n\nTo give you a clear idea of your options, I'd need a bit more information. Could you please tell me:\n\n* **What type of insurance policy do you have?** Is it a health insurance policy, a hospitalization policy, or something else?\n* **Does your policy specifically cover dengue?** \n* **What is your policy's maximum coverage limit for a single hospitalization?** \n\nOnce I have this information, I can assess whether your claim is likely to be approved and what documentation you'll need to gather. \n"

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoModel
model_name = "jinaai/jina-embeddings-v2-base-en"
model_kwargs ={'trust_remote_code': True}


c:\Users\Aakash Rajaraman\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embeddings_model = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-small-en",model_kwargs=model_kwargs)


In [2]:
from werkzeug.datastructures import ImmutableMultiDict

In [3]:
x = ImmutableMultiDict([('policy_number', ''), ('policyholder_name', ''), ('dob', ''), ('contact_number', ''), ('email', ''), ('address', ''), ('claim_amount', ''), ('date_of_treatment', ''), ('hospital_name', ''), ('hospital_address', ''), ('diagnosis', ''), ('treatment_description', ''), ('doctor_name', ''), ('doctor_contact', ''), ('policy_start_date', ''), ('policy_end_date', ''), ('has_previous_claims', 'yes'), ('supporting_documents', '')])

In [6]:
x.keys()

dict_keys(['policy_number', 'policyholder_name', 'dob', 'contact_number', 'email', 'address', 'claim_amount', 'date_of_treatment', 'hospital_name', 'hospital_address', 'diagnosis', 'treatment_description', 'doctor_name', 'doctor_contact', 'policy_start_date', 'policy_end_date', 'has_previous_claims', 'supporting_documents'])

In [1]:
import os
import google.generativeai as genai
from google.generativeai import caching
import datetime
import time
import dotenv
from vertexai.generative_models import Part

c:\Users\Aakash Rajaraman\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="cred.json"

In [9]:
path = r"../docs/health_form.pdf"

In [14]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        extracted_text = ""
        for page in pdf_reader.pages:
            text = page.extract_text()
            if text:
                print("t", text)
                extracted_text += text
        return extracted_text


In [15]:
print(extract_text_from_pdf(path))

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [4]:
paths =[
    "../docs/car insurance.pdf",
    "../docs/united india health insurance.pdf",
    "../docs/home insurance.pdf",
]

In [5]:
docs = [PyPDFLoader(url).load() for url in paths]
docs_list = [item for sublist in docs for item in sublist]

In [6]:

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)

In [7]:
ids = ["id"+str(i) for i in range(1,len(doc_splits)+1)]
print(ids)

['id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', 'id9', 'id10', 'id11', 'id12', 'id13', 'id14', 'id15', 'id16', 'id17', 'id18', 'id19', 'id20', 'id21', 'id22', 'id23', 'id24', 'id25', 'id26', 'id27', 'id28', 'id29', 'id30', 'id31', 'id32', 'id33', 'id34', 'id35', 'id36', 'id37', 'id38', 'id39', 'id40', 'id41', 'id42', 'id43', 'id44', 'id45', 'id46', 'id47', 'id48', 'id49', 'id50', 'id51', 'id52', 'id53', 'id54', 'id55', 'id56', 'id57', 'id58', 'id59', 'id60', 'id61', 'id62', 'id63', 'id64', 'id65', 'id66', 'id67', 'id68', 'id69', 'id70', 'id71', 'id72', 'id73', 'id74', 'id75', 'id76', 'id77', 'id78', 'id79', 'id80', 'id81', 'id82', 'id83', 'id84', 'id85', 'id86', 'id87', 'id88', 'id89', 'id90', 'id91', 'id92', 'id93', 'id94', 'id95', 'id96', 'id97', 'id98', 'id99', 'id100', 'id101', 'id102', 'id103', 'id104', 'id105', 'id106', 'id107', 'id108', 'id109', 'id110', 'id111', 'id112', 'id113', 'id114', 'id115', 'id116', 'id117', 'id118', 'id119', 'id120', 'id121', 'id122', 'id123', 

In [8]:
final_doc_splits = [i.page_content for i in doc_splits]

In [9]:
import chromadb
from langchain_chroma import Chroma
persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=ids, documents=final_doc_splits)

vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embeddings_model,
)

c:\Users\Aakash Rajaraman\AppData\Local\Programs\Python\Python311\Lib\site-packages\onnxruntime\capi\_pybind_state.py:26: UserWarning: Please install the 2019 Visual C++ runtime and then try again. If you've installed the runtime in a non-standard location (other than %SystemRoot%\System32), make sure it can be found by setting the correct path.
  warnings.warn("Please install the 2019 Visual C++ runtime and then try again. "


: 

In [1]:
example_claim = {
  "claimId": "DHC20240509001",
  "patientDetails": {
    "name": "Ranjit Sharma",
    "policyNumber": "HSI123456789",
    "dateOfBirth": "1985-06-15",
    "gender": "Male",
    "maritalStatus": "Married",
    "spouseName": "Anita Sharma",
    "contactNumber": "+91-9876543210",
    "email": "ranjit.sharma@example.com",
    "address": {
      "street": "15, MG Road",
      "city": "Bengaluru",
      "state": "Karnataka",
      "postalCode": "560001",
      "country": "India"
    }
  },
  "claimDetails": {
    "diagnosisCode": "A90",
    "diagnosisDescription": "Dengue fever [classical dengue]",
    "dateOfDiagnosis": "2024-05-01",
    "hospitalName": "Apollo Hospital, Bengaluru",
    "admissionDate": "2024-05-02",
    "dischargeDate": "2024-05-07",
    "totalBillAmount": 250000.00,
    "claimAmount": 225000.00,
    "claimDate": "2024-05-09"
  },
  "claimStatus": {
    "status": "REJECTED",
    "rejectionDate": "2024-05-15",
    "rejectionReasons": [
      "Pre-existing condition not disclosed",
      "Waiting period for vector-borne diseases not completed"
    ],
    "appealDeadline": "2024-06-14"
  },
  "insuranceCompany": {
    "name": "SBI Health Insurance",
    "contactNumber": "+91-1800-22-1111",
    "email": "customer.care@sbigeneral.in",
    "address": "Corporate Office, Fulcrum Building, 9th Floor, A & B Wing, Sahar Road, Andheri (East), Mumbai - 400099"
  },
  "policyDetails": {
    "policyProvider": "SBI Health Insurance",
    "policyPath": "docs/united india health insurance.pdf",
    "policyStartDate": "2020-01-01",
    "policyEndDate": "2025-01-01",
    "coverageAmount": 500000,
    "premiumAmount": 12000,
    "premiumFrequency": "Yearly"
  }
}


In [2]:
import claim_resolution
state_dict = {
    "type": "health",
    "claim_json":   example_claim 
}

c:\Users\Aakash Rajaraman\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Aakash Rajaraman\AppData\Local\Temp\ipykernel_3988\225560302.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  import claim_resolution


In [3]:
claim_resolution.generate_resolution(state_dict)

ValueError: ('connection should be an instance of lancedb.db.LanceTable, ', "got <class 'lancedb.db.LanceDBConnection'>")